<a href="https://colab.research.google.com/github/mini1115/TIL/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0%20%EA%B8%B0%EC%B6%9C/8%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from google.colab import files
uploaded = files.upload()

Saving piq.csv to piq.csv


In [15]:
import pandas as pd
pd.set_option('display.max_columns',None)
df = pd.read_csv('drinks.csv')

ans = df.groupby(['continent'],as_index=False)['beer_servings'].mean()
ans.sort_values('beer_servings',ascending=False)

# Europe

cond = df['continent']=='Europe'
df[cond][['beer_servings','country']].sort_values('beer_servings',ascending=False).head(5)
# 313

,beer_servings,country
45,361,Czech Republic
65,346,Germany
135,343,Poland
98,343,Lithuania
81,313,Ireland


In [37]:
import pandas as pd
df = pd.read_csv('tourist.csv')
df['rate'] = df['관광'] / (df['관광']+df['공무'] +df['사업'] +df['기타'])

ans = df.sort_values('rate',ascending=False)
ans.iloc[1,3]


203

In [44]:
import pandas as pd
df = pd.read_csv('chem.csv')
# df.head()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['new_co'] = scaler.fit_transform(df[['co']])
df['new_nmhc'] = scaler.fit_transform(df[['nmhc']])
print(round(-df['new_nmhc'].std()+df['new_co'].std(),3))

-0.017


In [79]:
import pandas as pd
train = pd.read_csv('churn_train.csv')
test = pd.read_csv('churn_test.csv')
# train.shape,test.shape
# test.isnull().sum().sum()
# print(train.describe(include='O'))
target = train.pop('TotalCharges')
# print(target)
# train.head()
train.pop('customerID')
test.pop('customerID')

combined = pd.concat([train,test])
combined_oh = pd.get_dummies(combined)
l = len(train)
train = combined_oh[:l]
test = combined_oh[l:]
# 946.9158838362998

# from sklearn.preprocessing import LabelEncoder
# cols = train.select_dtypes(include='O').columns
# for col in cols:
#   le = LabelEncoder()
#   train[col] = le.fit_transform(train[col])
#   test[col] = le.fit_transform(test[col])
# 951.0960435538027

cols = train.select_dtypes(exclude='O').columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])


from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)
# print(x_tr.shape,x_val.shape,y_tr.shape,y_val.shape)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=5,random_state=0)
rf.fit(x_tr,y_tr)
pred = rf.predict(x_val)

# import lightgbm as lgb
# lg = lgb.LGBMRegressor(random_state=0,verbose=-1)
# lg.fit(x_tr,y_tr)
# pred = lg.predict(x_val)

from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_val,pred)
print(score)

pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv',index=False)
print(pd.read_csv('result.csv'))

<ipython-input-79-522fa40b124d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cols] = scaler.fit_transform(train[cols])
<ipython-input-79-522fa40b124d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[cols] = scaler.transform(test[cols])


914.4499668658017
             pred
0     3448.711197
1      934.422528
2     3988.684410
3      937.972744
4     1299.263399
...           ...
1759   280.435865
1760  3949.363411
1761  3788.601598
1762  1043.372302
1763  3633.527343

[1764 rows x 1 columns]


In [93]:
import pandas as pd
df = pd.read_csv('churn.csv')

from statsmodels.formula.api import logit
formula = 'Churn ~ AccountWeeks+ ContractRenewal  +DataPlan +DataUsage  +CustServCalls  +DayMins  +DayCalls  +MonthlyCharge  +OverageFee  +RoamMins  '
model = logit(formula,data=df).fit()
ans = (model.pvalues <0.05)

new_formula = 'Churn ~ DataUsage+DayMins'
new_model = logit(new_formula,data=df).fit()
# new_model.params.sum()

import numpy as np
ans = np.exp(new_model.params['DataUsage']*5)
ans

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6


0.42798312548416284

In [100]:
import pandas as pd
df = pd.read_csv('piq.csv')
# df.head()
from statsmodels.formula.api import ols
model = ols('PIQ ~ Brain + Height+ Weight',data=df).fit()
model.params['Brain']

2.343130100951171